In [1]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import pandas as pd
import rich as rc
import os

In [3]:
data = pd.read_csv("../data/DicatJuanRana.csv", sep=";")

In [4]:
data.head()

,year,text
0,0,"Aunque su verdadero nombre era Cosme Pérez, fu..."
1,1617,"Según Cotarelo , a quien sigue Sáez Raposo , C..."
2,1621,Consta la lista de la compañía de Juan Bautist...
3,1624,Cosme Pérez aparece en la nómina de la compañí...
4,1634,"Según Cotarelo, Cosme Pérez pertenecía a la co..."


In [2]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY"),
)

In [9]:
prompt = PromptTemplate(
    input_variables=["text"],
    template="""
    Separa el siguiente texto en frases y devuelve cada frase en una línea separada:
    {text}""",
)

In [12]:
from tqdm.auto import tqdm

all_sentences = []
for i, row in tqdm(data.iterrows(), total=len(data)):
    text = row["text"]
    response = llm.invoke(prompt.format(text=text))
    sentences = response.content.split("\n")
    all_sentences.append(sentences)

  0%|          | 0/38 [00:00<?, ?it/s]

In [13]:
len(all_sentences)

38

In [14]:
data["sentences"] = all_sentences

In [18]:
data.to_csv(
    "../data/DicatJuanRana_w_sentences.csv",
    sep=";",
    index=False,
)